In [258]:
import cPickle, gzip, os, sys, time 
import timeit
import numpy as np
import theano
import theano.tensor as T
from matplotlib.pyplot import imshow

In [242]:
# %load utils.py
""" This file contains different utility functions that are not connected
in anyway to the networks presented in the tutorials, but rather help in
processing the outputs into a more understandable way.

For example ``tile_raster_images`` helps in generating a easy to grasp
image from a set of samples or weights.
"""


import numpy


def scale_to_unit_interval(ndar, eps=1e-8):
    """ Scales all values in the ndarray ndar to be between 0 and 1 """
    ndar = ndar.copy()
    ndar -= ndar.min()
    ndar *= 1.0 / (ndar.max() + eps)
    return ndar


def tile_raster_images(X, img_shape, tile_shape, tile_spacing=(0, 0),
                       scale_rows_to_unit_interval=True,
                       output_pixel_vals=True):
    """
    Transform an array with one flattened image per row, into an array in
    which images are reshaped and layed out like tiles on a floor.

    This function is useful for visualizing datasets whose rows are images,
    and also columns of matrices for transforming those rows
    (such as the first layer of a neural net).

    :type X: a 2-D ndarray or a tuple of 4 channels, elements of which can
    be 2-D ndarrays or None;
    :param X: a 2-D array in which every row is a flattened image.

    :type img_shape: tuple; (height, width)
    :param img_shape: the original shape of each image

    :type tile_shape: tuple; (rows, cols)
    :param tile_shape: the number of images to tile (rows, cols)

    :param output_pixel_vals: if output should be pixel values (i.e. int8
    values) or floats

    :param scale_rows_to_unit_interval: if the values need to be scaled before
    being plotted to [0,1] or not


    :returns: array suitable for viewing as an image.
    (See:`Image.fromarray`.)
    :rtype: a 2-d array with same dtype as X.

    """

    assert len(img_shape) == 2
    assert len(tile_shape) == 2
    assert len(tile_spacing) == 2

    # The expression below can be re-written in a more C style as
    # follows :
    #
    # out_shape    = [0,0]
    # out_shape[0] = (img_shape[0]+tile_spacing[0])*tile_shape[0] -
    #                tile_spacing[0]
    # out_shape[1] = (img_shape[1]+tile_spacing[1])*tile_shape[1] -
    #                tile_spacing[1]
    out_shape = [
        (ishp + tsp) * tshp - tsp
        for ishp, tshp, tsp in zip(img_shape, tile_shape, tile_spacing)
    ]

    if isinstance(X, tuple):
        assert len(X) == 4
        # Create an output numpy ndarray to store the image
        if output_pixel_vals:
            out_array = numpy.zeros((out_shape[0], out_shape[1], 4),
                                    dtype='uint8')
        else:
            out_array = numpy.zeros((out_shape[0], out_shape[1], 4),
                                    dtype=X.dtype)

        #colors default to 0, alpha defaults to 1 (opaque)
        if output_pixel_vals:
            channel_defaults = [0, 0, 0, 255]
        else:
            channel_defaults = [0., 0., 0., 1.]

        for i in xrange(4):
            if X[i] is None:
                # if channel is None, fill it with zeros of the correct
                # dtype
                dt = out_array.dtype
                if output_pixel_vals:
                    dt = 'uint8'
                out_array[:, :, i] = numpy.zeros(
                    out_shape,
                    dtype=dt
                ) + channel_defaults[i]
            else:
                # use a recurrent call to compute the channel and store it
                # in the output
                out_array[:, :, i] = tile_raster_images(
                    X[i], img_shape, tile_shape, tile_spacing,
                    scale_rows_to_unit_interval, output_pixel_vals)
        return out_array

    else:
        # if we are dealing with only one channel
        H, W = img_shape
        Hs, Ws = tile_spacing

        # generate a matrix to store the output
        dt = X.dtype
        if output_pixel_vals:
            dt = 'uint8'
        out_array = numpy.zeros(out_shape, dtype=dt)

        for tile_row in xrange(tile_shape[0]):
            for tile_col in xrange(tile_shape[1]):
                if tile_row * tile_shape[1] + tile_col < X.shape[0]:
                    this_x = X[tile_row * tile_shape[1] + tile_col]
                    if scale_rows_to_unit_interval:
                        # if we should scale values to be between 0 and 1
                        # do this by calling the `scale_to_unit_interval`
                        # function
                        this_img = scale_to_unit_interval(
                            this_x.reshape(img_shape))
                    else:
                        this_img = this_x.reshape(img_shape)
                    # add the slice to the corresponding position in the
                    # output array
                    c = 1
                    if output_pixel_vals:
                        c = 255
                    out_array[
                        tile_row * (H + Hs): tile_row * (H + Hs) + H,
                        tile_col * (W + Ws): tile_col * (W + Ws) + W
                    ] = this_img * c
        return out_array


In [243]:
#Load the data sets
f = gzip.open('mnist.pkl.gz','rb')
train_set, valid_set, test_set = cPickle.load(f)
f.close()

In [118]:
print train_set[0].shape, train_set[1].shape
print valid_set[0].shape, valid_set[1].shape
print test_set[0].shape, test_set[1].shape

print train_set[0]
print train_set[1]

#print 'train= ',train_set[1]
np.sqrt(784) #28
#print len(train_set)
#print train_set
#print train_set[0]
#print train_set[1]

(50000, 784) (50000,)
(10000, 784) (10000,)
(10000, 784) (10000,)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[5 0 4 ..., 8 4 8]


28.0

In [33]:
def shared_dataset(data_xy):
    data_x, data_y = data_xy
    shared_x = theano.shared(np.asarray(data_x, dtype=theano.config.floatX))
    shared_y = theano.shared(np.asarray(data_y, dtype=theano.config.floatX))
    return shared_x, T.cast(shared_y,'int32')

test_set_x, test_set_y = shared_dataset(test_set)
valid_set_x, valid_set_y = shared_dataset(valid_set)
train_set_x, train_set_y = shared_dataset(train_set)

batch_size = 500

data  = train_set_x[2*500:3*500]
label = train_set_y[2*500:3*500]


In [153]:
#Making sure if it runs in GPU
from theano import function, config, shared, sandbox
import theano.sandbox.cuda.basic_ops
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), 'float32'))
f = function([], sandbox.cuda.basic_ops.gpu_from_host(T.exp(x)))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in xrange(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
print("Numpy result is %s" % (numpy.asarray(r),))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

EnvironmentError: ('The following error happened while compiling the node', GpuFromHost(Elemwise{exp,no_inplace}.0), '\n', "You forced the use of gpu device 'gpu', but nvcc was not found. Set it in your PATH environment variable or set the Theano flags 'cuda.root' to its directory")

In [244]:
def load_data(dataset):
        
    #Load the datasets
    f = gzip.open(dataset, 'rb')
    train_set, valid_set, test_set = cPickle.load(f)
    f.close()
        
    def shared_dataset(data_xy, borrow=True):
            
        data_x, data_y = data_xy
        shared_x = theano.shared(np.asarray(data_x, dtype=theano.config.floatX), borrow=borrow)
        shared_y = theano.shared(np.asarray(data_y, dtype=theano.config.floatX), borrow=borrow)
        return shared_x, T.cast(shared_y,'int32')
        
    test_set_x, test_set_y = shared_dataset(test_set)
    valid_set_x, valid_set_y = shared_dataset(valid_set)
    train_set_x, train_set_y = shared_dataset(train_set)
        
    rval = [(train_set_x, train_set_y ), (valid_set_x, valid_set_y), (test_set_x, test_set_y)]
        
    return rval

In [267]:
class LogisticRegression:
    
    def __init__(self, input, n_in, n_out):
        
        self.W = theano.shared(value=np.zeros((n_in, n_out), dtype = theano.config.floatX), name='W', borrow=True)
        self.b = theano.shared(value=np.zeros((n_out,), dtype=theano.config.floatX), name='b', borrow=True)
        self.p_y_given_x = T.nnet.softmax(T.dot(input, self.W)+self.b)
        self.y_pred = T.argmax(self.p_y_given_x, axis =1)
        self.params = [self.W, self.b]
        
    def negative_log_likelihood(self, y):
        
        return -T.mean( T.log(self.p_y_given_x)[T.arange(y.shape[0]),y] )
    
    def errors(self, y):
        
        if y.ndim != self.y_pred.ndim:
            raise TypeError('y should have the same shape as self.y_pred', ('y', y.type, 'y_pred', self.y_pred_.type) )
            
        if y.dtype.startswith('int'):
            return T.mean(T.neq(self.y_pred, y))
        else:
            raise NotImplementedError()            
        
    
    def sgd_optimization_mnist(self, learning_rate = 0.13, n_epochs = 100, dataset='mnist.pkl.gz', batch_size=500):
        
        datasets = load_data(dataset)
        
        train_set_x , train_set_y = datasets[0]
        valid_set_x, valid_set_y = datasets[1]
        test_set_x, test_set_y = datasets[2]
        
        n_train_batches = train_set_x.get_value(borrow=True).shape[0]/ batch_size
        n_valid_batches = valid_set_x.get_value(borrow=True).shape[0]/ batch_size
        n_test_batches = test_set_x.get_value(borrow=True).shape[0]/batch_size
        
        print '...building the model'
        
        index = T.lscalar()
        print "index= ", index
        print "n_train_batches= ",n_train_batches
        
        x = T.matrix('x')
        y = T.ivector('y')
        
        classifier = LogisticRegression(input =x, n_in = 28*28, n_out=10)
        
        cost = classifier.negative_log_likelihood(y)
        
        test_model = theano.function( inputs = [index],
                                      outputs = classifier.errors(y),
                                      givens={
                                            x: test_set_x[index*batch_size:(index+1)*batch_size],
                                            y: test_set_y[index*batch_size:(index+1)*batch_size]
                                        }
                                    )
        
        validate_model = theano.function( inputs = [index],
                                      outputs = classifier.errors(y),
                                      givens={
                                            x: valid_set_x[index*batch_size:(index+1)*batch_size],
                                            y: valid_set_y[index*batch_size:(index+1)*batch_size]
                                        }
                                    )
        
        g_W = T.grad(cost=cost, wrt = classifier.W)
        g_b = T.grad(cost=cost, wrt = classifier.b)
        
        updates = [ (classifier.W, classifier.W - learning_rate*g_W),
                    (classifier.b, classifier.b - learning_rate*g_b)]
        
        train_model = theano.function(
                        inputs = [index],
                        outputs = cost,
                        updates = updates,
                        givens = {
                                x: train_set_x[index*batch_size:(index+1)*batch_size],
                                y: train_set_y[index*batch_size:(index+1)*batch_size]
                            }
                        )
    
        
        print '.....training the model'
        
        #early stopping parameters
        patience = 5000
        patience_increase = 2
        improvement_threshold = 0.995
        validation_frequency = min(n_train_batches, patience/2)
        
        print 'validation_frequency= ',validation_frequency
        
        best_validation_loss = np.inf
        test_score = 0.
        start_time = time.clock
        
        done_looping = False
        epoch = 0
        while(epoch < n_epochs) and (not done_looping): #n_epochs=100
            epoch = epoch +1
            for minibatch_index in xrange(n_train_batches): #n_train_batches=100
                
                #print "minibatch_index=",minibatch_index
                
                minibatch_avg_cost = train_model(minibatch_index)
                iter = (epoch -1)*n_train_batches + minibatch_index
                #print "iter=",iter
                
                if (iter+1) % validation_frequency == 0 :
                    validation_losses = [validate_model(i) for i in xrange(n_valid_batches)]
                    this_validation_loss = np.mean(validation_losses)
                    print ( 'epoch %i, minibatch avg. %i/%i, validation error %f %%' %
                              (epoch, minibatch_index+1, n_train_batches, this_validation_loss*100)
                          )
                    
                    if(False and epoch % 10 == 0):
                        image = Image.fromarray(
                        tile_raster_images(
                            X=classifier.W.get_value(borrow=True).T,
                            img_shape=(28, 28),
                            tile_shape=(10, 10),
                            tile_spacing=(1, 1) )
                        )
                        #imshow(np.asarray(image))
                        image.save('filters_at_epoch_%i.png' % epoch)
        
                    
                    if (this_validation_loss < best_validation_loss*improvement_threshold):
                        patience = max(patience, iter*patience_increase)

                    best_validation_loss = this_validation_loss

                    test_losses = [test_model(i) for i in xrange(n_test_batches)]
                    test_score = np.mean(test_losses)

                    print ( 'epoch %i, minibatch avg. %i/%i, test error of best model %f %%' %
                           (epoch, minibatch_index+1, n_train_batches, test_score*100.)
                          )
                
                if (patience<=iter):
                    done_looping = True
                    break
                
        end_time = time.clock()
        print('Optimization complete with best validation score of %f with test performance %f %%' %
                (best_validation_loss*100., test_score*100.) )
        print 'The code run for %d epochs, with %f seconds ' %(epoch, end_time)
        print >> sys.stderr #, ('The code for file '+ os.path.split(__file__)[1] + 'ran for %.1fs'%((end_time-start_time)))

#if __name__ == '__main__':    
#    sgd_optimization_mnist()
        

In [268]:
x = T.matrix('x')
logReg = LogisticRegression(x,n_in = 28*28, n_out=10)
logReg.sgd_optimization_mnist()

...building the model
index=  <TensorType(int64, scalar)>
n_train_batches=  100
.....training the model
validation_frequency=  100
epoch 1, minibatch avg. 100/100, validation error 11.890000 %
epoch 1, minibatch avg. 100/100, test error of best model 12.170000 %
epoch 2, minibatch avg. 100/100, validation error 10.520000 %
epoch 2, minibatch avg. 100/100, test error of best model 10.940000 %
epoch 3, minibatch avg. 100/100, validation error 9.940000 %
epoch 3, minibatch avg. 100/100, test error of best model 10.210000 %
epoch 4, minibatch avg. 100/100, validation error 9.480000 %
epoch 4, minibatch avg. 100/100, test error of best model 9.820000 %
epoch 5, minibatch avg. 100/100, validation error 9.230000 %
epoch 5, minibatch avg. 100/100, test error of best model 9.510000 %
epoch 6, minibatch avg. 100/100, validation error 9.100000 %
epoch 6, minibatch avg. 100/100, test error of best model 9.280000 %
epoch 7, minibatch avg. 100/100, validation error 8.820000 %
epoch 7, minibatch avg.

In [247]:
class HiddenLayer(object):
    def __init__(self, rng, input, n_in, n_out, W=None, b=None, activation=T.tanh):
        self.input = input
        
        if W is None:
            W_values = np.asarray(rng.uniform(
                                              low  = -np.sqrt(6./(n_in + n_out)),
                                              high =  np.sqrt(6./(n_in+n_out)),
                                              size = (n_in, n_out)
                                             ), dtype = theano.config.floatX)
            if activation == theano.tensor.nnet.sigmoid:
                W_value *= 4
                
            W = theano.shared(value=W_values, name='W', borrow=True)
            
        if b is None:
            b_value = np.zeros((n_out,), dtype=theano.config.floatX)
            b = theano.shared(value=b_value, name='b', borrow=True)
            
        self.W = W
        self.b = b
        
        lin_output = T.dot(input, self.W) + self.b
        self.output = (lin_output if activation is None else activation(lin_output) )
        self.params = [self.W, self.b]
        

In [273]:

class MLP(object):
    def __init__(self, rng, input, n_in, n_hidden, n_out):
        self.hiddenLayer = HiddenLayer(rng = rng, input = input, n_in=n_in, n_out=n_hidden, activation=T.tanh)
        self.logRegressionLayer = LogisticRegression(input=self.hiddenLayer.output, n_in=n_hidden, n_out= n_out)
        self.L1 = ( abs(self.hiddenLayer.W).sum() + abs(self.logRegressionLayer.W).sum() )
        self.L2_sqr = ( (self.hiddenLayer.W**2).sum() + (self.logRegressionLayer.W**2).sum() )
        self.negative_log_likelihood = ( self.logRegressionLayer.negative_log_likelihood )
        self.errors = self.logRegressionLayer.errors
        self.params = self.hiddenLayer.params + self.logRegressionLayer.params
        
          
    def test_mlp(self, learning_rate=0.01, L1_reg=0.00, L2_reg=0.0001, n_epochs=100,
             dataset='mnist.pkl.gz', batch_size=20, n_hidden=500):

        datasets = load_data(dataset)

        train_set_x, train_set_y = datasets[0]
        valid_set_x, valid_set_y = datasets[1]
        test_set_x, test_set_y = datasets[2]

        # compute number of minibatches for training, validation and testing
        n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
        n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
        n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size

        ######################
        # BUILD ACTUAL MODEL #
        ######################
        print '... building the model'

        # allocate symbolic variables for the data
        index = T.lscalar()  # index to a [mini]batch
        x = T.matrix('x')  # the data is presented as rasterized images
        y = T.ivector('y')  # the labels are presented as 1D vector of
                            # [int] labels

        rng = np.random.RandomState(1234)

        # construct the MLP class
        classifier = MLP(
            rng=rng,
            input=x,
            n_in=28 * 28,
            n_hidden=n_hidden,
            n_out=10
        )

        # the cost we minimize during training is the negative log likelihood of
        # the model plus the regularization terms (L1 and L2); cost is expressed
        # here symbolically
        cost = (
            classifier.negative_log_likelihood(y)
            + L1_reg * classifier.L1
            + L2_reg * classifier.L2_sqr
        )
       
        # compiling a Theano function that computes the mistakes that are made
        # by the model on a minibatch
        test_model = theano.function(
            inputs=[index],
            outputs=classifier.errors(y),
            givens={
                x: test_set_x[index * batch_size:(index + 1) * batch_size],
                y: test_set_y[index * batch_size:(index + 1) * batch_size]
            }
        )

        validate_model = theano.function(
            inputs=[index],
            outputs=classifier.errors(y),
            givens={
                x: valid_set_x[index * batch_size:(index + 1) * batch_size],
                y: valid_set_y[index * batch_size:(index + 1) * batch_size]
            }
        )

        # compute the gradient of cost with respect to theta (sotred in params)
        # the resulting gradients will be stored in a list gparams
        gparams = [T.grad(cost, param) for param in classifier.params]

        # specify how to update the parameters of the model as a list of
        # (variable, update expression) pairs

        updates = [
            (param, param - learning_rate * gparam)
            for param, gparam in zip(classifier.params, gparams)
        ]

        # compiling a Theano function `train_model` that returns the cost, but
        # in the same time updates the parameter of the model based on the rules
        # defined in `updates`
        train_model = theano.function(
            inputs=[index],
            outputs=cost,
            updates=updates,
            givens={
                x: train_set_x[index * batch_size: (index + 1) * batch_size],
                y: train_set_y[index * batch_size: (index + 1) * batch_size]
            }
        )
     
        ###############
        # TRAIN MODEL #
        ###############
        print '... training'

        # early-stopping parameters
        patience = 10000  # look as this many examples regardless
        patience_increase = 2  # wait this much longer when a new best is
                               # found
        improvement_threshold = 0.995  # a relative improvement of this much is
                                       # considered significant
        validation_frequency = min(n_train_batches, patience / 2)
                                      # go through this many
                                      # minibatche before checking the network
                                      # on the validation set; in this case we
                                      # check every epoch

        best_validation_loss = np.inf
        best_iter = 0
        test_score = 0.
        start_time = timeit.default_timer()

        epoch = 0
        done_looping = False

        while (epoch < n_epochs) and (not done_looping):
            epoch = epoch + 1
            for minibatch_index in xrange(n_train_batches):

                minibatch_avg_cost = train_model(minibatch_index)
                # iteration number
                iter = (epoch - 1) * n_train_batches + minibatch_index

                if (iter + 1) % validation_frequency == 0:
                    # compute zero-one loss on validation set
                    validation_losses = [validate_model(i) for i
                                         in xrange(n_valid_batches)]
                    this_validation_loss = np.mean(validation_losses)

                    print(
                        'epoch %i, minibatch %i/%i, validation error %f %%' %
                        (
                            epoch,
                            minibatch_index + 1,
                            n_train_batches,
                            this_validation_loss * 100.
                        )
                    )
                    
                    if(True and epoch % 10 == 0):
                        image = Image.fromarray(
                        tile_raster_images(
                            X=classifier.hiddenLayer.W.get_value(borrow=True).T,
                            img_shape=(28, 28),
                            tile_shape=(10, 10),
                            tile_spacing=(1, 1) )
                        )
                        #imshow(np.asarray(image))
                        image.save('filters_at_epoch_%i.png' % epoch)

                    # if we got the best validation score until now
                    if this_validation_loss < best_validation_loss:
                        #improve patience if loss improvement is good enough
                        if (this_validation_loss < best_validation_loss *improvement_threshold):
                            patience = max(patience, iter * patience_increase)

                        best_validation_loss = this_validation_loss
                        best_iter = iter

                        # test it on the test set
                        test_losses = [test_model(i) for i
                                       in xrange(n_test_batches)]
                        test_score = np.mean(test_losses)

                        print(('epoch %i, minibatch %i/%i, test error of '
                               'best model %f %%') %
                              (epoch, minibatch_index + 1, n_train_batches,
                               test_score * 100.))

                if patience <= iter:
                    done_looping = True
                    print "patience = ", patience, " iter = ", iter
                    break

        end_time = timeit.default_timer()
        print(('Optimization complete. Best validation score of %f %% '
               'obtained at iteration %i, with test performance %f %%') %
              (best_validation_loss * 100., best_iter + 1, test_score * 100.))
        print >> sys.stderr #, ('The code for file ' + os.path.split(__file__)[1] +
                            #' ran for %.2fm' % ((end_time - start_time) / 60.))

In [ ]:
x = T.matrix('x')
rng = np.random.RandomState(1234)
mlp = MLP(rng, input=x, n_in=28 * 28, n_hidden=100, n_out=10)
mlp.test_mlp()

... building the model
... training
epoch 1, minibatch 2500/2500, validation error 9.620000 %
epoch 1, minibatch 2500/2500, test error of best model 10.090000 %
epoch 2, minibatch 2500/2500, validation error 8.610000 %
epoch 2, minibatch 2500/2500, test error of best model 8.740000 %

In [ ]:
class LeNetConvPoolLayer(object):
    """Pool Layer of a convolutional network """

    def __init__(self, rng, input, filter_shape, image_shape, poolsize=(2, 2)):

        assert image_shape[1] == filter_shape[1]
        self.input = input

        # there are "num input feature maps * filter height * filter width"
        # inputs to each hidden unit
        fan_in = numpy.prod(filter_shape[1:])
        # each unit in the lower layer receives a gradient from:
        # "num output feature maps * filter height * filter width" /
        #   pooling size
        fan_out = (filter_shape[0] * numpy.prod(filter_shape[2:]) /
                   numpy.prod(poolsize))
        # initialize weights with random weights
        W_bound = numpy.sqrt(6. / (fan_in + fan_out))
        self.W = theano.shared(
            numpy.asarray(
                rng.uniform(low=-W_bound, high=W_bound, size=filter_shape),
                dtype=theano.config.floatX
            ),
            borrow=True
        )

        # the bias is a 1D tensor -- one bias per output feature map
        b_values = numpy.zeros((filter_shape[0],), dtype=theano.config.floatX)
        self.b = theano.shared(value=b_values, borrow=True)

        # convolve input feature maps with filters
        conv_out = conv.conv2d(
            input=input,
            filters=self.W,
            filter_shape=filter_shape,
            image_shape=image_shape
        )

        # downsample each feature map individually, using maxpooling
        pooled_out = downsample.max_pool_2d(
            input=conv_out,
            ds=poolsize,
            ignore_border=True
        )

        # add the bias term. Since the bias is a vector (1D array), we first
        # reshape it to a tensor of shape (1, n_filters, 1, 1). Each bias will
        # thus be broadcasted across mini-batches and feature map
        # width & height
        self.output = T.tanh(pooled_out + self.b.dimshuffle('x', 0, 'x', 'x'))

        # store parameters of this layer
        self.params = [self.W, self.b]

        # keep track of model input
        self.input = input

In [ ]:
nkerns=[20, 50]   #Number of kernel filter in layer0 and layer1
learning_rate=0.01
#L1_reg=0.00
#L2_reg=0.0001
n_epochs=100
#dataset='mnist.pkl.gz' 
batch_size=50
#n_hidden=500
dataset='mnist.pkl.gz'
    
x = T.matrix('x')   # the data is presented as rasterized images
y = T.ivector('y')  # the labels are presented as 1D vector of
                        # [int] labels
    
datasets = load_data(dataset)

train_set_x, train_set_y = datasets[0]
valid_set_x, valid_set_y = datasets[1]
test_set_x,  test_set_y  = datasets[2]

# compute number of minibatches for training, validation and testing
n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
n_test_batches  = test_set_x.get_value(borrow=True).shape[0] / batch_size

######################
# BUILD ACTUAL MODEL #
######################
print '... building the model'

# Reshape matrix of rasterized images of shape (batch_size, 28 * 28)
# to a 4D tensor, compatible with our LeNetConvPoolLayer
# (28, 28) is the size of MNIST images.
layer0_input = x.reshape((batch_size, 1, 28, 28))

# Construct the first convolutional pooling layer:
# filtering reduces the image size to (28-5+1 , 28-5+1) = (24, 24)
# maxpooling reduces this further to (24/2, 24/2) = (12, 12)
# 4D output tensor is thus of shape (batch_size, nkerns[0], 12, 12)
layer0 = LeNetConvPoolLayer(
        rng,
        input=layer0_input,
        image_shape=(batch_size, 1, 28, 28),
        filter_shape=(nkerns[0], 1, 5, 5),
        poolsize=(2, 2)
    )

# Construct the second convolutional pooling layer
# filtering reduces the image size to (12-5+1, 12-5+1) = (8, 8)
# maxpooling reduces this further to (8/2, 8/2) = (4, 4)
# 4D output tensor is thus of shape (batch_size, nkerns[1], 4, 4)
layer1 = LeNetConvPoolLayer(
        rng,
        input=layer0.output,
        image_shape=(batch_size, nkerns[0], 12, 12),
        filter_shape=(nkerns[1], nkerns[0], 5, 5),
        poolsize=(2, 2)
    )

# the HiddenLayer being fully-connected, it operates on 2D matrices of
# shape (batch_size, num_pixels) (i.e matrix of rasterized images).
# This will generate a matrix of shape (batch_size, nkerns[1] * 4 * 4),
# or (500, 50 * 4 * 4) = (500, 800) with the default values.
layer2_input = layer1.output.flatten(2)

# construct a fully-connected sigmoidal layer
layer2 = HiddenLayer(
        rng,
        input=layer2_input,
        n_in=nkerns[1] * 4 * 4,
        n_out=500,
        activation=T.tanh
    )

# classify the values of the fully-connected sigmoidal layer
layer3 = LogisticRegression(input=layer2.output, n_in=500, n_out=10)

# the cost we minimize during training is the NLL of the model
cost = layer3.negative_log_likelihood(y)
    
index = T.lscalar()  # index to a [mini]batch
    
# create a function to compute the mistakes that are made by the model
test_model = theano.function(
        [index],
        layer3.errors(y),
        givens={
            x: test_set_x[index * batch_size: (index + 1) * batch_size],
            y: test_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

validate_model = theano.function(
        [index],
        layer3.errors(y),
        givens={
            x: valid_set_x[index * batch_size: (index + 1) * batch_size],
            y: valid_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

# create a list of all model parameters to be fit by gradient descent
params = layer3.params + layer2.params + layer1.params + layer0.params

# create a list of gradients for all model parameters
grads = T.grad(cost, params)

# train_model is a function that updates the model parameters by
# SGD Since this model has many parameters, it would be tedious to
# manually create an update rule for each model parameter. We thus
# create the updates list by automatically looping over all
# (params[i], grads[i]) pairs.
updates = [
        (param_i, param_i - learning_rate * grad_i)
        for param_i, grad_i in zip(params, grads)
    ]

train_model = theano.function(
        [index],
        cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            y: train_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )
    
###############
# TRAIN MODEL #
###############
print '... training'

# early-stopping parameters
patience = 10000  # look as this many examples regardless
patience_increase = 2  # wait this much longer when a new best is
                               # found
improvement_threshold = 0.995  # a relative improvement of this much is
                                       # considered significant
validation_frequency = min(n_train_batches, patience / 2)
                                      # go through this many
                                      # minibatche before checking the network
                                      # on the validation set; in this case we
                                      # check every epoch

best_validation_loss = np.inf
best_iter = 0
test_score = 0.
start_time = timeit.default_timer()

epoch = 0
done_looping = False

while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in xrange(n_train_batches):

            minibatch_avg_cost = train_model(minibatch_index)
            # iteration number
            iter = (epoch - 1) * n_train_batches + minibatch_index

            if (iter + 1) % validation_frequency == 0:
                # compute zero-one loss on validation set
                validation_losses = [validate_model(i) for i
                                         in xrange(n_valid_batches)]
                this_validation_loss = np.mean(validation_losses)

                print(
                        'epoch %i, minibatch %i/%i, validation error %f %%' %
                        (
                            epoch,
                            minibatch_index + 1,
                            n_train_batches,
                            this_validation_loss * 100.
                        )
                    )

                # if we got the best validation score until now
                if this_validation_loss < best_validation_loss:
                    #improve patience if loss improvement is good enough
                    if (this_validation_loss < best_validation_loss *improvement_threshold):
                        patience = max(patience, iter * patience_increase)

                    best_validation_loss = this_validation_loss
                    best_iter = iter

                    # test it on the test set
                    test_losses = [test_model(i) for i
                                       in xrange(n_test_batches)]
                    test_score = np.mean(test_losses)

                    print(('epoch %i, minibatch %i/%i, test error of '
                               'best model %f %%') %
                              (epoch, minibatch_index + 1, n_train_batches,
                               test_score * 100.))

            if patience <= iter:
                done_looping = True
                print "patience = ", patience, " iter = ", iter
                break

end_time = timeit.default_timer()
print(('Optimization complete. Best validation score of %f %% '
               'obtained at iteration %i, with test performance %f %%') %
              (best_validation_loss * 100., best_iter + 1, test_score * 100.))
print >> sys.stderr #, ('The code for file ' + os.path.split(__file__)[1] +
                            #' ran for %.2fm' % ((end_time - start_time) / 60.))